![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# 4.Training and Reusing Text Classification Models

**Relevant blogpost:** https://towardsdatascience.com/text-classification-in-spark-nlp-with-bert-and-universal-sentence-encoders-e644d618ca32

In [0]:
import json
import os
import string
import pandas as pd
import numpy as np

import sparknlp
from sparknlp.base import *
from sparknlp.annotator import *
from sparknlp.util import *

import pyspark.sql.functions as F
from pyspark.ml import Pipeline, PipelineModel
from pyspark.sql import functions as F


pd.set_option('display.max_columns', None)  
pd.set_option('display.expand_frame_repr', False)
pd.set_option('max_colwidth', 100)

print('sparknlp.version : ',sparknlp.version())

spark

sparknlp_jsl.version : 4.2.4
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.1.2 
 Master 
 spark://10.139.64.5:7077 
 AppName 
 Databricks Shell

## Using Pretrained ClassifierDL and SentimentDL models

In [0]:
fake_classifier = ClassifierDLModel.pretrained('classifierdl_use_fakenews', 'en') \
                .setInputCols(["sentence_embeddings"]) \
                .setOutputCol("class")

classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[ | ][OK!]

fake_news classifier is trained on `https://raw.githubusercontent.com/joolsa/fake_real_news_dataset/master/fake_or_real_news.csv.zip`

In [0]:
fake_classifier.getClasses()

Out[166]: ['FAKE', 'REAL']

In [0]:
documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained(name="tfhub_use",lang="en") \
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      use,
      fake_classifier
  ])

empty_data = spark.createDataFrame([[""]]).toDF("text")

fake_clf_model = nlpPipeline.fit(empty_data)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/spam_ham_dataset.csv
  
dbutils.fs.cp("file:/databricks/driver/spam_ham_dataset.csv", "dbfs:/") 

Out[168]: True

In [0]:
fake_lp_pipeline = LightPipeline(fake_clf_model)

text = 'BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump'

fake_lp_pipeline.annotate(text)

Out[169]: {'document': ['BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump'],
 'sentence_embeddings': ['BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump'],
 'class': ['FAKE']}

In [0]:
sample_data = spark.createDataFrame([[text]]).toDF("text")

sample_data.show(truncate=False)

+-------------------------------------------------------------------------------------------------+
text |
+-------------------------------------------------------------------------------------------------+
BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump|
+-------------------------------------------------------------------------------------------------+

In [0]:
pred = fake_clf_model.transform(sample_data)


In [0]:
pred.show()

+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence_embeddings| class|
+--------------------+--------------------+--------------------+--------------------+
BREAKING: Leaked ...|[{document, 0, 96...|[{sentence_embedd...|[{category, 0, 96...|
+--------------------+--------------------+--------------------+--------------------+

In [0]:
pred.select('text','class.result').show(truncate=False)

+-------------------------------------------------------------------------------------------------+------+
text |result|
+-------------------------------------------------------------------------------------------------+------+
BREAKING: Leaked Picture Of Obama Being Dragged Before A Judge In Handcuffs For Wiretapping Trump|[FAKE]|
+-------------------------------------------------------------------------------------------------+------+

you can find more samples here >> `https://github.com/KaiDMML/FakeNewsNet/tree/master/dataset`

## Generic classifier function

In [0]:
def get_clf_lp(model_name, sentiment_dl=False, pretrained=True):

  documentAssembler = DocumentAssembler()\
      .setInputCol("text")\
      .setOutputCol("document")

  use = UniversalSentenceEncoder.pretrained(lang="en") \
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")


  if pretrained:

    if sentiment_dl:

      document_classifier = SentimentDLModel.pretrained(model_name, 'en') \
                .setInputCols(["sentence_embeddings"]) \
                .setOutputCol("class")
    else:
      document_classifier = ClassifierDLModel.pretrained(model_name, 'en') \
                .setInputCols(["sentence_embeddings"]) \
                .setOutputCol("class")

  else:

    if sentiment_dl:

      document_classifier = SentimentDLModel.load(model_name) \
                .setInputCols(["sentence_embeddings"]) \
                .setOutputCol("class")
    else:
      document_classifier = ClassifierDLModel.load(model_name) \
                .setInputCols(["sentence_embeddings"]) \
                .setOutputCol("class")

  print ('classes:',document_classifier.getClasses())

  nlpPipeline = Pipeline(stages=[
                  documentAssembler, 
                  use,
                  document_classifier
  ])

  empty_data = spark.createDataFrame([[""]]).toDF("text")

  clf_pipelineFit = nlpPipeline.fit(empty_data)

  clf_lp_pipeline = LightPipeline(clf_pipelineFit)

  return clf_lp_pipeline


In [0]:
clf_lp_pipeline = get_clf_lp('classifierdl_use_trec50')

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_trec50 download started this may take some time.
Approximate size to download 21.2 MB
[ | ][OK!]
classes: [' ENTY_color', ' ENTY_techmeth', ' DESC_manner', ' NUM_volsize', ' ENTY_letter', ' NUM_temp', ' ENTY_body', ' NUM_count', ' ENTY_instru', ' NUM_period', ' NUM_speed', ' DESC_reason', ' ENTY_symbol', ' ENTY_event', ' HUM_desc', ' NUM_perc', ' ENTY_dismed', ' NUM_ord', ' HUM_gr', ' LOC_mount', ' ABBR_abb', ' DESC_desc', ' NUM_dist', ' HUM_title', ' ENTY_lang', ' ENTY_sport', ' ENTY_plant', ' NUM_code', ' NUM_other', ' ENTY_word', ' ENTY_animal', ' ENTY_substance', ' ENTY_veh', ' ENTY_product', ' LOC_state', ' ENTY_religion', ' ENTY_currency', ' NUM_date', ' LOC_country', ' ENTY_cremat', ' NUM_money', ' LOC_other', ' DESC_def', ' LOC_city', ' HUM_ind', ' ENTY_other', ' ENTY_termeq', ' ENTY_food', ' ABBR_exp', ' NUM_weight']

trained on the TREC datasets:

Classify open-domain, fact-based questions into one of the following broad semantic categories: 

```Abbreviation, Description, Entities, Human Beings, Locations or Numeric Values.```

In [0]:
text = 'What was the number of member nations of the U.N. in 2000?'

clf_lp_pipeline.annotate(text)['class']

Out[176]: [' NUM_count']

In [0]:
clf_lp_pipeline.fullAnnotate(text)[0]['class'][0].result

Out[177]: ' NUM_count'

In [0]:
clf_lp_pipeline.fullAnnotate(text)[0]['class'][0].metadata

Out[178]: {' ENTY_dismed': '3.768739E-22', ' ENTY_product': '2.4015744E-24', ' ENTY_techmeth': '1.5787039E-22', ' NUM_speed': '7.948464E-23', ' NUM_volsize': '2.5315113E-25', ' LOC_state': '6.3784123E-25', ' NUM_code': '1.4549451E-25', ' NUM_count': '0.9992601', ' ENTY_food': '1.3031208E-24', ' ENTY_animal': '1.6743833E-24', ' NUM_period': '6.8075115E-21', ' ENTY_religion': '5.9194734E-23', ' LOC_country': '5.3062683E-21', ' LOC_mount': '3.2177816E-25', ' ENTY_termeq': '9.790085E-26', ' ENTY_color': '1.1446835E-22', ' ENTY_lang': '6.333391E-24', ' ENTY_sport': '8.0773835E-25', ' DESC_def': '2.4284432E-27', ' HUM_gr': '4.4863106E-21', ' ENTY_symbol': '4.1271923E-25', ' ENTY_currency': '8.156541E-29', ' ENTY_veh': '5.414701E-22', ' LOC_other': '5.5141072E-11', ' ENTY_word': '5.3265024E-23', ' NUM_temp': '2.0907158E-23', ' NUM_dist': '1.2542656E-24', ' DESC_desc': '1.0926973E-12', ' DESC_manner': '9.258374E-23', ' NUM_ord': '2.2395288E-25', ' NUM_other': '3.9771262E-27', ' DESC_reason': '1.1718967E-6', ' NUM_weight': '1.5373857E-24', ' ENTY_instru': '5.9354656E-21', ' ENTY_letter': '1.1453239E-25', ' ENTY_event': '3.706315E-25', ' ENTY_substance': '6.890844E-25', ' ABBR_exp': '5.6048268E-24', ' ENTY_body': '6.423101E-23', ' ENTY_other': '7.378E-4', ' NUM_money': '1.6745677E-25', ' LOC_city': '4.7003377E-22', ' NUM_date': '5.2122506E-16', ' NUM_perc': '6.3761288E-24', ' ABBR_abb': '7.101014E-26', ' ENTY_plant': '5.543376E-24', ' HUM_title': '1.0681953E-24', ' ENTY_cremat': '1.1165376E-24', ' HUM_ind': '8.063818E-7', ' HUM_desc': '4.3701275E-23', 'sentence': '0'}

In [0]:
text = 'What animal was the first mammal successfully cloned from adult cells?'

clf_lp_pipeline.annotate(text)['class']

Out[179]: [' HUM_ind']

In [0]:
clf_lp_pipeline = get_clf_lp('classifierdl_use_cyberbullying')


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_cyberbullying download started this may take some time.
Approximate size to download 21.3 MB
[ | ][OK!]
classes: ['sexism', 'neutral', 'racism']

In [0]:
text ='RT @EBeisner @ahall012 I agree with you!! I would rather brush my teeth with sandpaper then watch football with a girl!!'

clf_lp_pipeline.annotate(text)['class']

Out[181]: ['sexism']

In [0]:
clf_lp_pipeline = get_clf_lp('classifierdl_use_fakenews')


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_fakenews download started this may take some time.
Approximate size to download 21.4 MB
[ | ][OK!]
classes: ['FAKE', 'REAL']

In [0]:
text ='Donald Trump a KGB Spy? 11/02/2016 In today’s video, Christopher Greene of AMTV reports Hillary Clinton campaign accusation that Donald Trump is a KGB spy is about as weak and baseless a claim as a Salem witch hunt or McCarthy era trial. It’s only because Hillary Clinton is losing that she is lobbing conspiracy theory. Citizen Quasar The way I see it, one of two things will happen: 1. Trump will win by a landslide but the election will be stolen via electronic voting, just like I have been predicting for over a decade, and the American People will accept the skewed election results just like they accept the TSA into their crotches. 2. Somebody will bust a cap in Hillary’s @$$ killing her and the election will be postponed. Follow AMTV!'

clf_lp_pipeline.annotate(text)['class']


Out[183]: ['FAKE']

In [0]:
text ='Sen. Marco Rubio (R-Fla.) is adding a veteran New Hampshire political operative to his team as he continues mulling a possible 2016 presidential bid, the latest sign that he is seriously preparing to launch a campaign later this year.Jim Merrill, who worked for former GOP presidential nominee Mitt Romney and ran his 2008 and 2012 New Hampshire primary campaigns, joined Rubio’s fledgling campaign on Monday, aides to the senator said.Merrill will be joining Rubio’s Reclaim America PAC to focus on Rubio’s New Hampshire and broader Northeast political operations."Marco has always been well received in New Hampshire, and should he run for president, he would be very competitive there," Terry Sullivan, who runs Reclaim America, said in a statement. "Jim certainly knows how to win in New Hampshire and in the Northeast, and will be a great addition to our team at Reclaim America.”News of Merrill’s hire was first reported by The New York Times.'

clf_lp_pipeline.annotate(text)['class']

Out[184]: ['REAL']

In [0]:
sentiment_lp_pipeline = get_clf_lp('sentimentdl_use_twitter', sentiment_dl=True)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[ | ][OK!]
classes: ['positive', 'negative']

In [0]:
text ='I am SO happy the news came out in time for my birthday this weekend! My inner 7-year-old cannot WAIT!'

sentiment_lp_pipeline.annotate(text)['class']

Out[186]: ['positive']

In [0]:
sentiment_lp_pipeline = get_clf_lp('classifierdl_use_emotion', sentiment_dl=False)


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classifierdl_use_emotion download started this may take some time.
Approximate size to download 21.3 MB
[ | ][OK!]
classes: ['joy', 'fear', 'surprise', 'sadness']

In [0]:
sentiment_lp_pipeline.annotate(text)['class']

Out[188]: ['surprise']

## ClassiferDL with Word Embeddings and Text Preprocessing

### Load Dataset

In [0]:
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_train.csv
! wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/news_category_test.csv
  
dbutils.fs.cp("file:/databricks/driver/news_category_train.csv", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/news_category_test.csv", "dbfs:/")

Out[189]: True

In [0]:
%sh cd /databricks/driver/ && ls -lt 

total 224472
-rw-r--r-- 1 root root 1504408 Jan 18 21:20 news_category_test.csv.2
-rw-r--r-- 1 root root 24032125 Jan 18 21:20 news_category_train.csv.2
-rw-r--r-- 1 root root 5502589 Jan 18 21:19 spam_ham_dataset.csv.2
drwxr-xr-x 2 root root 4096 Jan 18 21:15 ganglia
drwxr-xr-x 2 root root 4096 Jan 18 21:02 logs
-rw-r--r-- 1 root root 188402 Jan 18 20:44 title_conference.csv.1
-rw-r--r-- 1 root root 296833 Jan 18 20:42 toxic_test.snappy.parquet
-rw-r--r-- 1 root root 2767307 Jan 18 20:42 toxic_train.snappy.parquet
-rw-r--r-- 1 root root 1504408 Jan 18 20:16 news_category_test.csv.1
-rw-r--r-- 1 root root 24032125 Jan 18 20:16 news_category_train.csv.1
-rw-r--r-- 1 root root 5502589 Jan 18 20:15 spam_ham_dataset.csv.1
-rw-r--r-- 1 root root 188402 Jan 18 20:06 title_conference.csv
-rw-r--r-- 1 root root 1504408 Jan 18 19:37 news_category_test.csv
-rw-r--r-- 1 root root 24032125 Jan 18 19:37 news_category_train.csv
-rw-r--r-- 1 root root 5502589 Jan 18 19:36 spam_ham_dataset.csv
drwxr-xr-x 3 root root 4096 Jan 18 19:27 eventlogs
-r-xr-xr-x 1 root root 813069 Jan 18 19:25 preload_class.lst
drwxr-xr-x 2 root root 4096 Jan 18 19:25 conf
-rw-r--r-- 1 root root 32715164 May 2 2020 aclimdb_test.csv
-rw-r--r-- 1 root root 32715164 May 2 2020 aclimdb_test.csv.1
-rw-r--r-- 1 root root 33497180 May 2 2020 aclimdb_train.csv
-rw-r--r-- 1 root root 33497180 May 2 2020 aclimdb_train.csv.1

In [0]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("/news_category_train.csv")

trainDataset.show(truncate=50)

+--------+--------------------------------------------------+
category| description|
+--------+--------------------------------------------------+
Business| Short sellers, Wall Street's dwindling band of...|
Business| Private investment firm Carlyle Group, which h...|
Business| Soaring crude prices plus worries about the ec...|
Business| Authorities have halted oil export flows from ...|
Business| Tearaway world oil prices, toppling records an...|
Business| Stocks ended slightly higher on Friday but sta...|
Business| Assets of the nation's retail money market mut...|
Business| Retail sales bounced back a bit in July, and n...|
Business|" After earning a PH.D. in Sociology, Danny Baz...|
Business| Short sellers, Wall Street's dwindling band o...|
Business| Soaring crude prices plus worries about the e...|
Business| OPEC can do nothing to douse scorching oil pr...|
Business| Non OPEC oil exporters should consider increa...|
Business| WASHINGTON/NEW YORK (Reuters) - The auction fo...|
Business| The dollar tumbled broadly on Friday after da...|
Business|If you think you may need to help your elderly ...|
Business|The purchasing power of kids is a big part of w...|
Business|There is little cause for celebration in the st...|
Business|The US trade deficit has exploded 19 to a recor...|
Business|Oil giant Shell could be bracing itself for a t...|
+--------+--------------------------------------------------+
only showing top 20 rows

In [0]:
trainDataset.count()


Out[192]: 120000

In [0]:
from pyspark.sql.functions import col

trainDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
category|count|
+--------+-----+
 World|30000|
Sci/Tech|30000|
 Sports|30000|
Business|30000|
+--------+-----+

In [0]:
testDataset = spark.read \
      .option("header", True) \
      .csv("/news_category_test.csv")

testDataset.show(truncate=50)

+--------+--------------------------------------------------+
category| description|
+--------+--------------------------------------------------+
Business|Unions representing workers at Turner Newall ...|
Sci/Tech| TORONTO, Canada A second team of rocketeers...|
Sci/Tech| A company founded by a chemistry researcher at...|
Sci/Tech| It's barely dawn when Mike Fitzpatrick starts ...|
Sci/Tech| Southern California's smog fighting agency wen...|
Sci/Tech|"The British Department for Education and Skill...|
Sci/Tech|"confessed author of the Netsky and Sasser viru...|
Sci/Tech|\\FOAF/LOAF and bloom filters have a lot of in...|
Sci/Tech|"Wiltshire Police warns about ""phishing"" afte...|
Sci/Tech|In its first two years, the UK's dedicated card...|
Sci/Tech| A group of technology companies including Tex...|
Sci/Tech| Apple Computer Inc.&lt;AAPL.O&gt; on Tuesday ...|
Sci/Tech| Free Record Shop, a Dutch music retail chain,...|
Sci/Tech|A giant 100km colony of ants which has been di...|
Sci/Tech| "Dolphin groups, or ""pods""|
Sci/Tech|Tyrannosaurus rex achieved its massive size due...|
Sci/Tech| Scientists have discovered irregular lumps be...|
Sci/Tech| ESAs Mars Express has relayed pictures from o...|
Sci/Tech|When did life begin? One evidential clue stems ...|
Sci/Tech|update Earnings per share rise compared with a ...|
+--------+--------------------------------------------------+
only showing top 20 rows

In [0]:
testDataset.groupBy("category") \
    .count() \
    .orderBy(col("count").desc()) \
    .show()

+--------+-----+
category|count|
+--------+-----+
 World| 1900|
Sci/Tech| 1900|
 Sports| 1900|
Business| 1900|
+--------+-----+

In [0]:
# if we want to split the dataset
'''(trainData, testData) = trainDataset.randomSplit([0.7, 0.3], seed = 100)
print("Train Dataset Count: " + str(trainData.count()))
print("Test Dataset Count: " + str(testData.count()))'''


Out[196]: '(trainData, testData) = trainDataset.randomSplit([0.7, 0.3], seed = 100)\nprint("Train Dataset Count: " + str(trainData.count()))\nprint("Test Dataset Count: " + str(testData.count()))'

In [0]:
document_assembler = DocumentAssembler() \
      .setInputCol("description") \
      .setOutputCol("document")

tokenizer = Tokenizer() \
      .setInputCols(["document"]) \
      .setOutputCol("token")
    
normalizer = Normalizer() \
      .setInputCols(["token"]) \
      .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

lemma = LemmatizerModel.pretrained('lemma_antbnc') \
      .setInputCols(["cleanTokens"]) \
      .setOutputCol("lemma")


lemma_antbnc download started this may take some time.
Approximate size to download 907.6 KB
[ | ][OK!]

### with Glove 100d embeddings

In [0]:
%fs mkdirs dbfs:/clf_glove_logs

res4: Boolean = true

In [0]:

glove_embeddings = WordEmbeddingsModel.pretrained("glove_100d") \
      .setInputCols(["document",'lemma'])\
      .setOutputCol("embeddings")\
      .setCaseSensitive(False)

embeddingsSentence = SentenceEmbeddings() \
      .setInputCols(["document", "embeddings"]) \
      .setOutputCol("sentence_embeddings") \
      .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("category")\
      .setMaxEpochs(2)\
      .setBatchSize(64) \
      .setLr(5e-3) \
      .setDropout(0.5)\
      .setEnableOutputLogs(True)\
      .setOutputLogsPath('dbfs:/clf_glove_logs')

clf_pipeline = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            embeddingsSentence,
            classsifierdl])

glove_100d download started this may take some time.
Approximate size to download 145.3 MB
[ | ][OK!]

In [0]:
'''
default classifierDL params:

maxEpochs -> 10,
lr -> 5e-3f,
dropout -> 0.5f,
batchSize -> 64,
enableOutputLogs -> false,
verbose -> Verbose.Silent.id,
validationSplit -> 0.0f,
outputLogsPath -> ""
'''

Out[199]: '\ndefault classifierDL params:\n\nmaxEpochs -> 10,\nlr -> 5e-3f,\ndropout -> 0.5f,\nbatchSize -> 64,\nenableOutputLogs -> false,\nverbose -> Verbose.Silent.id,\nvalidationSplit -> 0.0f,\noutputLogsPath -> ""\n'

In [0]:
# Train (2 min for 2 epochs)

clf_pipelineModel = clf_pipeline.fit(trainDataset)

In [0]:
%sh cd /dbfs/clf_glove_logs/ && ls -lt 

total 5
-rwxrwxrwx 1 root root 251 Jan 18 21:21 ClassifierDLApproach_c9335482fadb.log
-rwxrwxrwx 1 root root 248 Jan 18 20:17 ClassifierDLApproach_2e90162f1943.log
-rwxrwxrwx 1 root root 246 Jan 18 19:39 ClassifierDLApproach_8f4e0a789b3b.log
-rwxrwxrwx 1 root root 247 Jan 18 17:46 ClassifierDLApproach_3116a9a8c042.log
-rwxrwxrwx 1 root root 251 Jan 18 15:52 ClassifierDLApproach_a5f7c28bed0a.log
-rwxrwxrwx 1 root root 250 Jan 18 15:37 ClassifierDLApproach_1cbacf770d39.log
-rwxrwxrwx 1 root root 247 Jan 8 18:49 ClassifierDLApproach_b22b3b8f3599.log
-rwxrwxrwx 1 root root 242 Oct 10 16:39 ClassifierDLApproach_0538f27c7737.log
-rwxrwxrwx 1 root root 247 Feb 8 2022 ClassifierDLApproach_2f4af8feed66.log

In [0]:
%sh cat /dbfs/clf_glove_logs/* 

Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 7.30s - loss: 1627.6475 - acc: 0.8691 - batches: 1875
Epoch 1/2 - 8.18s - loss: 1609.324 - acc: 0.88045 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 9.98s - loss: 1647.9044 - acc: 0.86836666 - batches: 1875
Epoch 1/2 - 9.52s - loss: 1620.5863 - acc: 0.88215834 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 11.02s - loss: 1651.4069 - acc: 0.8674333 - batches: 1875
Epoch 1/2 - 10.15s - loss: 1627.0297 - acc: 0.88105 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 20.63s - loss: 1640.8435 - acc: 0.8707 - batches: 1875
Epoch 1/2 - 19.17s - loss: 1609.2996 - acc: 0.8815917 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 8.19s - loss: 1632.2864 - acc: 0.86825 - batches: 1875
Epoch 1/2 - 7.46s - loss: 1609.3625 - acc: 0.88009167 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 10.50s - loss: 1659.227 - acc: 0.867275 - batches: 1875
Epoch 1/2 - 9.65s - loss: 1630.1442 - acc: 0.881775 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 10.33s - loss: 1629.8367 - acc: 0.86914164 - batches: 1875
Epoch 1/2 - 10.04s - loss: 1595.8134 - acc: 0.8814333 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 7.35s - loss: 1646.0 - acc: 0.86884165 - batches: 1875
Epoch 1/2 - 6.77s - loss: 1628.3286 - acc: 0.88096666 - batches: 1875
Training started - epochs: 2 - learning_rate: 0.005 - batch_size: 64 - training_examples: 120000 - classes: 4
Epoch 0/2 - 11.21s - loss: 1633.7614 - acc: 0.86819166 - batches: 1875
Epoch 1/2 - 10.89s - loss: 1613.5706 - acc: 0.8811333 - batches: 1875

In [0]:
# get the predictions on test Set

preds = clf_pipelineModel.transform(testDataset)

preds.select('category','description',"class.result").show(10, truncate=80)

+--------+--------------------------------------------------------------------------------+----------+
category| description| result|
+--------+--------------------------------------------------------------------------------+----------+
Business|Unions representing workers at Turner Newall say they are 'disappointed' af...|[Business]|
Sci/Tech| TORONTO, Canada A second team of rocketeers competing for the #36;10 mil...|[Sci/Tech]|
Sci/Tech| A company founded by a chemistry researcher at the University of Louisville ...|[Sci/Tech]|
Sci/Tech| It's barely dawn when Mike Fitzpatrick starts his shift with a blur of color...|[Sci/Tech]|
Sci/Tech| Southern California's smog fighting agency went after emissions of the bovin...|[Sci/Tech]|
Sci/Tech|"The British Department for Education and Skills (DfES) recently launched a "...|[Sci/Tech]|
Sci/Tech|"confessed author of the Netsky and Sasser viruses, is responsible for 70 per...|[Sci/Tech]|
Sci/Tech|\\FOAF/LOAF and bloom filters have a lot of interesting properties for socia...|[Sci/Tech]|
Sci/Tech|"Wiltshire Police warns about ""phishing"" after its fraud squad chief was ta...|[Sci/Tech]|
Sci/Tech|In its first two years, the UK's dedicated card fraud unit, has recovered 36,...|[Sci/Tech]|
+--------+--------------------------------------------------------------------------------+----------+
only showing top 10 rows

In [0]:
# due to bug in cluster mode (https://github.com/JohnSnowLabs/spark-nlp/issues/857) , as a workaround, you can just save the fitted model and then load back from dbfs and then transform on the test set. 
clf_pipelineModel.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/ClassifierDL_wordemb_g100d')
classsifierdlmodel_loaded = ClassifierDLModel.load('dbfs:/databricks/driver/models/ClassifierDL_wordemb_g100d')


clf_pipeline_pred = Pipeline(
    stages=[document_assembler, 
            tokenizer,
            normalizer,
            stopwords_cleaner, 
            lemma, 
            glove_embeddings,
            clf_pipelineModel.stages[-2],
            classsifierdlmodel_loaded])

empty_data = spark.createDataFrame([[""]]).toDF("description")

result = clf_pipeline_pred.fit(empty_data).transform(testDataset)

In [0]:
preds_df = result.select('category','description',"class.result").toPandas()

# The result is an array since in Spark NLP you can have multiple sentences.
# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['category']))


precision recall f1-score support

 Business 0.86 0.81 0.83 2014
 Sci/Tech 0.84 0.85 0.84 1872
 Sports 0.95 0.95 0.95 1907
 World 0.86 0.91 0.89 1807

 accuracy 0.88 7600
 macro avg 0.88 0.88 0.88 7600
weighted avg 0.88 0.88 0.88 7600

### with Bert Embeddings

In [0]:
document_assembler = DocumentAssembler() \
                .setInputCol("description") \
                .setOutputCol("document")

tokenizer = Tokenizer() \
                .setInputCols(["document"]) \
                .setOutputCol("token")
      
bert_embeddings = BertEmbeddings().pretrained(name='small_bert_L4_256', lang='en') \
                .setInputCols(["document",'token'])\
                .setOutputCol("embeddings")

embeddingsSentence = SentenceEmbeddings() \
                .setInputCols(["document", "embeddings"]) \
                .setOutputCol("sentence_embeddings") \
                .setPoolingStrategy("AVERAGE")

classsifierdl = ClassifierDLApproach()\
                .setInputCols(["sentence_embeddings"])\
                .setOutputCol("class")\
                .setLabelColumn("category")\
                .setMaxEpochs(10)\
                .setLr(0.001)\
                .setBatchSize(8)\
                .setEnableOutputLogs(True)\
                .setOutputLogsPath('dbfs:/clf_bert_logs')

bert_clf_pipeline = Pipeline(stages=[document_assembler,
                                     tokenizer,
                                     bert_embeddings,
                                     embeddingsSentence,
                                     classsifierdl])

small_bert_L4_256 download started this may take some time.
Approximate size to download 40.5 MB
[ | ][OK!]

In [0]:
# remove the existing logs

! rm -r /root/annotator_logs

rm: cannot remove '/root/annotator_logs': No such file or directory

In [0]:
# training will take some time due to Bert (use GPU runtime when possible)

bert_clf_pipelineModel = bert_clf_pipeline.fit(trainDataset)

In [0]:
%sh cd /dbfs/clf_bert_logs/ && ls -lt 

total 5
-rwxrwxrwx 1 root root 824 Jan 18 21:39 ClassifierDLApproach_393c04fdc101.log
-rwxrwxrwx 1 root root 833 Jan 18 20:35 ClassifierDLApproach_277cc0b5bbb1.log
-rwxrwxrwx 1 root root 823 Jan 18 19:57 ClassifierDLApproach_2ff3eb4259e4.log
-rwxrwxrwx 1 root root 823 Jan 18 17:59 ClassifierDLApproach_3c5b7e1632d7.log
-rwxrwxrwx 1 root root 253 Feb 4 2022 ClassifierDLApproach_3bf510f9e159.log

In [0]:
%sh cat /dbfs/clf_bert_logs/* 

Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/10 - 51.26s - loss: 13316.869 - acc: 0.85573334 - batches: 15000
Epoch 1/10 - 53.15s - loss: 13111.651 - acc: 0.869975 - batches: 15000
Epoch 2/10 - 50.40s - loss: 13015.457 - acc: 0.8773583 - batches: 15000
Epoch 3/10 - 49.07s - loss: 12962.077 - acc: 0.88266665 - batches: 15000
Epoch 4/10 - 51.60s - loss: 12914.85 - acc: 0.8865917 - batches: 15000
Epoch 5/10 - 51.42s - loss: 12872.899 - acc: 0.8895083 - batches: 15000
Epoch 6/10 - 51.73s - loss: 12834.163 - acc: 0.89238334 - batches: 15000
Epoch 7/10 - 50.52s - loss: 12796.696 - acc: 0.89475834 - batches: 15000
Epoch 8/10 - 50.58s - loss: 12765.013 - acc: 0.89685833 - batches: 15000
Epoch 9/10 - 50.21s - loss: 12738.552 - acc: 0.8986167 - batches: 15000
Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/10 - 49.91s - loss: 13335.299 - acc: 0.855975 - batches: 15000
Epoch 1/10 - 48.68s - loss: 13120.338 - acc: 0.8703833 - batches: 15000
Epoch 2/10 - 48.95s - loss: 13020.424 - acc: 0.8771167 - batches: 15000
Epoch 3/10 - 48.48s - loss: 12957.696 - acc: 0.88184166 - batches: 15000
Epoch 4/10 - 50.16s - loss: 12916.484 - acc: 0.885625 - batches: 15000
Epoch 5/10 - 48.50s - loss: 12878.053 - acc: 0.8884417 - batches: 15000
Epoch 6/10 - 48.61s - loss: 12841.42 - acc: 0.8913417 - batches: 15000
Epoch 7/10 - 49.39s - loss: 12810.181 - acc: 0.8936 - batches: 15000
Epoch 8/10 - 48.43s - loss: 12783.97 - acc: 0.8955333 - batches: 15000
Epoch 9/10 - 48.68s - loss: 12758.514 - acc: 0.897325 - batches: 15000
Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/10 - 51.98s - loss: 13350.956 - acc: 0.8552333 - batches: 15000
Epoch 1/10 - 51.24s - loss: 13136.743 - acc: 0.86975 - batches: 15000
Epoch 2/10 - 51.09s - loss: 13025.483 - acc: 0.8770083 - batches: 15000
Epoch 3/10 - 50.69s - loss: 12953.085 - acc: 0.881825 - batches: 15000
Epoch 4/10 - 51.12s - loss: 12895.694 - acc: 0.8860083 - batches: 15000
Epoch 5/10 - 50.27s - loss: 12847.632 - acc: 0.8893667 - batches: 15000
Epoch 6/10 - 53.71s - loss: 12808.43 - acc: 0.89205 - batches: 15000
Epoch 7/10 - 54.83s - loss: 12777.251 - acc: 0.8941583 - batches: 15000
Epoch 8/10 - 53.62s - loss: 12748.877 - acc: 0.89641666 - batches: 15000
Epoch 9/10 - 53.26s - loss: 12724.57 - acc: 0.8979333 - batches: 15000
Training started - epochs: 2 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/2 - 42.96s - loss: 19865.465 - acc: 0.32381666 - batches: 15000
Epoch 1/2 - 46.85s - loss: 19814.223 - acc: 0.32563335 - batches: 15000
Training started - epochs: 10 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/10 - 38.61s - loss: 13311.776 - acc: 0.8552333 - batches: 15000
Epoch 1/10 - 38.72s - loss: 13094.07 - acc: 0.8702 - batches: 15000
Epoch 2/10 - 38.01s - loss: 13000.639 - acc: 0.87731665 - batches: 15000
Epoch 3/10 - 39.03s - loss: 12937.432 - acc: 0.88226664 - batches: 15000
Epoch 4/10 - 38.70s - loss: 12886.709 - acc: 0.8860083 - batches: 15000
Epoch 5/10 - 37.93s - loss: 12841.292 - acc: 0.88915 - batches: 15000
Epoch 6/10 - 37.21s - loss: 12798.919 - acc: 0.891575 - batches: 15000
Epoch 7/10 - 38.57s - loss: 12760.591 - acc: 0.89381665 - batches: 15000
Epoch 8/10 - 37.43s - loss: 12729.11 - acc: 0.8959 - batches: 15000
Epoch 9/10 - 37.18s - loss: 12701.382 - acc: 0.89770836 - batches: 15000

In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

preds = bert_clf_pipelineModel.transform(testDataset)

preds_df = preds.select('category','description',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

precision recall f1-score support

 Business 0.82 0.84 0.83 1900
 Sci/Tech 0.86 0.83 0.85 1900
 Sports 0.93 0.96 0.95 1900
 World 0.89 0.86 0.88 1900

 accuracy 0.87 7600
 macro avg 0.87 0.87 0.87 7600
weighted avg 0.87 0.87 0.87 7600

#### testDataset Parameter

In [0]:
embedding_pipeline = Pipeline(stages=[document_assembler,
                                     tokenizer,
                                     bert_embeddings,
                                     embeddingsSentence])

In [0]:
test_data = embedding_pipeline.fit(testDataset).transform(testDataset)

In [0]:
test_data.write.overwrite().parquet('test_data.parquet')

In [0]:
classsifierdl = ClassifierDLApproach()\
                .setInputCols(["sentence_embeddings"])\
                .setOutputCol("class")\
                .setLabelColumn("category")\
                .setMaxEpochs(10)\
                .setLr(0.001)\
                .setBatchSize(8)\
                .setEnableOutputLogs(True)\
                .setTestDataset("test_data.parquet")\
                .setOutputLogsPath('clf_logs')

clf_pipeline = Pipeline(stages=[document_assembler,
                                tokenizer,
                                bert_embeddings,
                                embeddingsSentence,
                                classsifierdl])

In [0]:
%%time
clf_model = clf_pipeline.fit(trainDataset)

In [0]:
from sklearn.metrics import classification_report

preds = clf_model.transform(testDataset)

preds_df = preds.select('category','description',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['category'], preds_df['result']))

### Getting prediction from Trained model

In [0]:
from sparknlp.base import LightPipeline

light_model = LightPipeline(clf_pipelineModel)

In [0]:
text='''
Fearing the fate of Italy, the centre-right government has threatened to be merciless with those who flout tough restrictions. 
As of Wednesday it will also include all shops being closed across Greece, with the exception of supermarkets. Banks, pharmacies, pet-stores, mobile phone stores, opticians, bakers, mini-markets, couriers and food delivery outlets are among the few that will also be allowed to remain open.
'''
result = light_model.annotate(text)

result['class']

Out[216]: ['Business']

In [0]:
light_model.annotate('the soccer games will be postponed.')

Out[217]: {'lemma': ['soccer', 'game', 'postpone'],
 'document': ['the soccer games will be postponed.'],
 'normalized': ['the', 'soccer', 'games', 'will', 'be', 'postponed'],
 'sentence_embeddings': ['the soccer games will be postponed.'],
 'cleanTokens': ['soccer', 'games', 'postponed'],
 'token': ['the', 'soccer', 'games', 'will', 'be', 'postponed', '.'],
 'class': ['Sports'],
 'embeddings': ['soccer', 'game', 'postpone']}

## ClassifierDL with Universal Sentence Embeddings

In [0]:
%fs mkdirs dbfs:/clf_use_logs

res3: Boolean = true

In [0]:
# actual content is inside description column
document = DocumentAssembler()\
      .setInputCol("description")\
      .setOutputCol("document")
    
# we can also use sentece detector here if we want to train on and get predictions for each sentence

use = UniversalSentenceEncoder.pretrained()\
     .setInputCols(["document"])\
     .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("category")\
      .setMaxEpochs(5)\
      .setLr(0.001)\
      .setBatchSize(8)\
      .setEnableOutputLogs(True)\
      .setOutputLogsPath('dbfs:/clf_use_logs')

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
# 5 epochs takes around 8 min

use_pipelineModel = use_clf_pipeline.fit(trainDataset)

In [0]:
%sh cd  /dbfs/clf_use_logs/ && ls -l


total 4
-rwxrwxrwx 1 root root 463 Jan 18 20:41 ClassifierDLApproach_1c04c9f52857.log
-rwxrwxrwx 1 root root 461 Jan 18 18:04 ClassifierDLApproach_7fd7a2d21889.log
-rwxrwxrwx 1 root root 467 Jan 18 15:43 ClassifierDLApproach_80ccd326ebeb.log
-rwxrwxrwx 1 root root 464 Oct 10 16:50 ClassifierDLApproach_99f47a5282fc.log
-rwxrwxrwx 1 root root 462 Jan 8 18:54 ClassifierDLApproach_ae9c3cff1971.log
-rwxrwxrwx 1 root root 462 Feb 8 2022 ClassifierDLApproach_d375b9652a4d.log
-rwxrwxrwx 1 root root 460 Jan 18 20:03 ClassifierDLApproach_ff79577dff7b.log

In [0]:
%sh cat /dbfs/clf_use_logs/*

Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/5 - 50.39s - loss: 12932.341 - acc: 0.8826917 - batches: 15000
Epoch 1/5 - 51.33s - loss: 12771.584 - acc: 0.8914667 - batches: 15000
Epoch 2/5 - 51.19s - loss: 12704.802 - acc: 0.89598334 - batches: 15000
Epoch 3/5 - 49.36s - loss: 12654.659 - acc: 0.8996083 - batches: 15000
Epoch 4/5 - 49.81s - loss: 12618.718 - acc: 0.90205 - batches: 15000
Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/5 - 39.34s - loss: 12940.258 - acc: 0.88261664 - batches: 15000
Epoch 1/5 - 38.19s - loss: 12799.002 - acc: 0.8912 - batches: 15000
Epoch 2/5 - 39.96s - loss: 12735.095 - acc: 0.89590836 - batches: 15000
Epoch 3/5 - 40.18s - loss: 12694.752 - acc: 0.899425 - batches: 15000
Epoch 4/5 - 39.92s - loss: 12665.023 - acc: 0.901975 - batches: 15000
Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/5 - 48.54s - loss: 12909.378 - acc: 0.88236666 - batches: 15000
Epoch 1/5 - 48.56s - loss: 12755.24 - acc: 0.8918667 - batches: 15000
Epoch 2/5 - 47.96s - loss: 12688.837 - acc: 0.89609164 - batches: 15000
Epoch 3/5 - 48.16s - loss: 12645.203 - acc: 0.89934164 - batches: 15000
Epoch 4/5 - 49.76s - loss: 12608.391 - acc: 0.90199167 - batches: 15000
Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/5 - 34.67s - loss: 12979.269 - acc: 0.8815167 - batches: 15000
Epoch 1/5 - 34.33s - loss: 12817.842 - acc: 0.891275 - batches: 15000
Epoch 2/5 - 33.84s - loss: 12760.203 - acc: 0.8955917 - batches: 15000
Epoch 3/5 - 33.76s - loss: 12715.922 - acc: 0.8987333 - batches: 15000
Epoch 4/5 - 33.94s - loss: 12675.6045 - acc: 0.9016333 - batches: 15000
Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/5 - 34.70s - loss: 12890.45 - acc: 0.88225836 - batches: 15000
Epoch 1/5 - 33.96s - loss: 12740.813 - acc: 0.892475 - batches: 15000
Epoch 2/5 - 34.00s - loss: 12684.14 - acc: 0.8964083 - batches: 15000
Epoch 3/5 - 33.78s - loss: 12648.879 - acc: 0.899675 - batches: 15000
Epoch 4/5 - 35.00s - loss: 12614.699 - acc: 0.90205836 - batches: 15000
Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/5 - 90.09s - loss: 12858.197 - acc: 0.88205 - batches: 15000
Epoch 1/5 - 93.15s - loss: 12700.624 - acc: 0.89110833 - batches: 15000
Epoch 2/5 - 80.61s - loss: 12645.851 - acc: 0.895875 - batches: 15000
Epoch 3/5 - 79.56s - loss: 12609.686 - acc: 0.899375 - batches: 15000
Epoch 4/5 - 80.09s - loss: 12578.779 - acc: 0.90219164 - batches: 15000
Training started - epochs: 5 - learning_rate: 0.001 - batch_size: 8 - training_examples: 120000 - classes: 4
Epoch 0/5 - 49.72s - loss: 12882.013 - acc: 0.88168335 - batches: 15000
Epoch 1/5 - 52.82s - loss: 12723.423 - acc: 0.892025 - batches: 15000
Epoch 2/5 - 53.29s - loss: 12651.68 - acc: 0.89624166 - batches: 15000
Epoch 3/5 - 50.02s - loss: 12611.896 - acc: 0.8991 - batches: 15000
Epoch 4/5 - 49.30s - loss: 12580.973 - acc: 0.901525 - batches: 15000

### Saving & loading back the trained model

In [0]:
use_pipelineModel.stages

Out[136]: [DocumentAssembler_022a08e8bca4,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 ClassifierDLModel_bb4faeb2e8ad]

In [0]:
use_pipelineModel.stages[2].write().overwrite().save('/databricks/driver/models/ClassifierDL_USE_e5')

In [0]:
classsifierdlmodel = ClassifierDLModel.load('dbfs:/databricks/driver/models/ClassifierDL_USE_e5')

In [0]:
clf_lp = get_clf_lp('dbfs:/databricks/driver/models/ClassifierDL_USE_e5', sentiment_dl=False,  pretrained=False)

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]
classes: ['World', 'Sci/Tech', 'Sports', 'Business']

In [0]:
clf_lp.annotate(text)['class']

Out[140]: ['Business']

# SentimentDL Classifier

see also here >> `https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/classification/SentimentDL_train_multiclass_sentiment_classifier.ipynb`

In [0]:
!wget -q aclimdb_train.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sentiment-corpus/aclimdb/aclimdb_train.csv
!wget -q aclimdb_test.csv https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/sentiment-corpus/aclimdb/aclimdb_test.csv
  
dbutils.fs.cp("file:/databricks/driver/aclimdb_train.csv", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/aclimdb_test.csv", "dbfs:/")

Out[141]: True

In [0]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("/aclimdb_train.csv")

trainDataset.show(10)

+--------------------+--------+
 text| label|
+--------------------+--------+
This is an Excell...|positive|
The Sarah Silverm...|positive|
"Prom Night" is a...|negative|
So often a band w...|positive|
"Pet Sematary" is...|positive|
I watched the fil...|negative|
Boy this movie ha...|negative|
Checking the spoi...|negative|
Despite its rathe...|positive|
Absolute masterpi...|positive|
+--------------------+--------+
only showing top 10 rows

In [0]:
testDataset = spark.read \
      .option("header", True) \
      .csv("/aclimdb_test.csv")

testDataset.show(10)

+--------------------+--------+
 text| label|
+--------------------+--------+
The Second Woman ...|negative|
In my opinion the...|positive|
I am listening to...|positive|
Before I speak my...|negative|
ManBearPig is a p...|positive|
A buddy and I wen...|negative|
It is incredible ...|negative|
Dire! Dismal! Awf...|negative|
HLOTS was an outs...|positive|
This is just one ...|negative|
+--------------------+--------+
only showing top 10 rows

In [0]:
# actual content is inside description column
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

use = UniversalSentenceEncoder.pretrained() \
   .setInputCols(["document"])\
   .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
sentimentdl = SentimentDLApproach()\
  .setInputCols(["sentence_embeddings"])\
  .setOutputCol("class")\
  .setLabelColumn("label")\
  .setMaxEpochs(5)\
  .setEnableOutputLogs(True)

pipeline = Pipeline(stages = [
    document,
    use,
    sentimentdl
    ])


tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
pipelineModel = pipeline.fit(trainDataset)

In [0]:
result = pipelineModel.transform(testDataset)

result_df = result.select('text','label',"class.result").toPandas()

result_df.head(10)

Out[146]:

,text,label,result
0,The Second Woman is about the story of a mysterious man who lost his wife in an accident and now...,negative,[negative]
1,"In my opinion the directing, editing, lighting, and acting(minus Franco) were very good. I must ...",positive,[positive]
2,"I am listening to Istanbul, intent, my eyes closed: At first there is a gentle breeze And the le...",positive,[positive]
3,"Before I speak my piece, I would like to make a few things clear: 1)I am a chick who's not into ...",negative,[positive]
4,ManBearPig is a pretty funny episode of South Park.It spoofs Al Gore and his speeches on Global ...,positive,[positive]
5,A buddy and I went to see this movie when it came out in 1980. It was playing in a huge theater ...,negative,[negative]
6,It is incredible that there were two films with the same story released in 2005. This one came o...,negative,[negative]
7,"Dire! Dismal! Awful! Laughable! Disappointing!<br /><br />Right, your trapped in ""The Cave"" with...",negative,[negative]
8,"HLOTS was an outstanding series, its what NYPD Blue will never be, on HLOTS the plots are real, ...",positive,[positive]
9,This is just one of those films which cannot justify much of anything that happens. These people...,negative,[negative]


# Multilabel Classifier

see also here >> `https://github.com/JohnSnowLabs/spark-nlp-workshop/blob/master/jupyter/training/english/classification/MultiClassifierDL_train_multi_label_toxic_classifier.ipynb`

In [0]:
!curl -O 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/toxic_comments/toxic_train.snappy.parquet'
!curl -O 'https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/classifier-dl/toxic_comments/toxic_test.snappy.parquet'

dbutils.fs.cp("file:/databricks/driver/toxic_train.snappy.parquet", "dbfs:/")
dbutils.fs.cp("file:/databricks/driver/toxic_test.snappy.parquet", "dbfs:/")

% Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0100 2702k 100 2702k 0 0 6894k 0 --:--:-- --:--:-- --:--:-- 6894k
 % Total % Received % Xferd Average Speed Time Time Time Current
 Dload Upload Total Spent Left Speed
 0 0 0 0 0 0 0 0 --:--:-- --:--:-- --:--:-- 0100 289k 100 289k 0 0 2787k 0 --:--:-- --:--:-- --:--:-- 2787k
Out[2]: True

In [0]:
trainDataset = spark.read.parquet("/toxic_train.snappy.parquet").repartition(120)
testDataset = spark.read.parquet("/toxic_test.snappy.parquet").repartition(10)

In [0]:
# Let's use shrink to remove new lines in the comments
document = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")\
  .setCleanupMode("shrink")

# Here we use the state-of-the-art Universal Sentence Encoder model from TF Hub
embeddings = UniversalSentenceEncoder.pretrained() \
  .setInputCols(["document"])\
  .setOutputCol("sentence_embeddings")

# We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes
# We will use only 5 Epochs but feel free to increase it on your own dataset
multiClassifier = MultiClassifierDLApproach()\
  .setInputCols("sentence_embeddings")\
  .setOutputCol("category")\
  .setLabelColumn("labels")\
  .setBatchSize(128)\
  .setMaxEpochs(5)\
  .setLr(1e-3)\
  .setThreshold(0.5)\
  .setShufflePerEpoch(False)\
  .setEnableOutputLogs(True)\
  .setValidationSplit(0.1)

pipeline = Pipeline(
    stages = [
        document,
        embeddings,
        multiClassifier
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
pipelineModel = pipeline.fit(trainDataset)

In [0]:
preds = pipelineModel.transform(testDataset)
preds_df = preds.select('text','labels',"category.result").toPandas()
preds_df.head(10)

Out[151]:

,text,labels,result
0,Vegan \n\nWhat in the hell does all that junk have to do with photos? 68.54.163.153,[toxic],[toxic]
1,Fight Club! F**k Yeeaaaaahh!!!,"[toxic, obscene]","[toxic, obscene]"
2,"""\n\n Little quick on the trigger, ain't'cha bud? \n\nYou know, if you're not even going to give...",[toxic],[toxic]
3,Your user page indicates you're a left-wing liberalist bastard! Why is in Wikipedia so much left...,"[toxic, obscene, insult]",[toxic]
4,""" See all the many Google links, titled"""" Wikipedia sucks.""""\n""",[toxic],[toxic]
5,"""\n\n LOL \n\nLOL. Seriously, """"BryanFromPalatine"""" ? My IP resolves to Cologne, Germany, where ...",[toxic],[toxic]
6,is it because it is of my naked mum having sex with me? i am indeed a faggot who sucks his dads ...,"[toxic, severe_toxic, obscene, insult, identity_hate]","[toxic, severe_toxic, insult, obscene]"
7,")\na cowards site, that must stop changing things he has no right to change.\nYOU HAVE NO RIGHT ...","[toxic, obscene]",[toxic]
8,"blow me, criticism IS constructive. \n\nBlow me Geoff.\n\nI came here to find out about monkeywr...","[toxic, obscene, insult]","[toxic, insult, obscene]"
9,"On account of the project deciding to ignore history, I hereby quit. Good bye. Oh, and Tim Vicke...","[toxic, obscene, insult]","[toxic, insult, obscene]"


## Multilabel Classifier with Bert Embeddings

In [0]:
# Let's use shrink to remove new lines in the comments
document = DocumentAssembler()\
              .setInputCol("text")\
              .setOutputCol("document")\
              .setCleanupMode("shrink")

bert_sent = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512')\
              .setInputCols(["document"])\
              .setOutputCol("sentence_embeddings")

# We will use MultiClassifierDL built by using Bidirectional GRU and CNNs inside TensorFlow that supports up to 100 classes
# We will use only 5 Epochs but feel free to increase it on your own dataset

multiClassifier = MultiClassifierDLApproach()\
              .setInputCols("sentence_embeddings")\
              .setOutputCol("category")\
              .setLabelColumn("labels")\
              .setBatchSize(128)\
              .setMaxEpochs(5)\
              .setLr(1e-3)\
              .setThreshold(0.5)\
              .setShufflePerEpoch(False)\
              .setEnableOutputLogs(True)\
              .setValidationSplit(0.1)

pipeline = Pipeline(stages = [document,
                              bert_sent,
                              multiClassifier])

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
%%time
pipelineModel = pipeline.fit(trainDataset)

CPU times: user 1.37 s, sys: 575 ms, total: 1.95 s
Wall time: 7min 2s

In [0]:
preds = pipelineModel.transform(testDataset)

In [0]:
preds_df = preds.select('text','labels',"category.result").toPandas()

In [0]:
preds_df.head(10)

Out[8]:

,text,labels,result
0,Vegan \n\nWhat in the hell does all that junk have to do with photos? 68.54.163.153,[toxic],"[toxic, obscene]"
1,Fight Club! F**k Yeeaaaaahh!!!,"[toxic, obscene]","[toxic, obscene]"
2,"""\n\n Little quick on the trigger, ain't'cha bud? \n\nYou know, if you're not even going to give...",[toxic],[toxic]
3,Your user page indicates you're a left-wing liberalist bastard! Why is in Wikipedia so much left...,"[toxic, obscene, insult]",[toxic]
4,""" See all the many Google links, titled"""" Wikipedia sucks.""""\n""",[toxic],[toxic]
5,"""\n\n LOL \n\nLOL. Seriously, """"BryanFromPalatine"""" ? My IP resolves to Cologne, Germany, where ...",[toxic],"[toxic, obscene]"
6,is it because it is of my naked mum having sex with me? i am indeed a faggot who sucks his dads ...,"[toxic, severe_toxic, obscene, insult, identity_hate]","[toxic, severe_toxic, insult, obscene]"
7,")\na cowards site, that must stop changing things he has no right to change.\nYOU HAVE NO RIGHT ...","[toxic, obscene]",[toxic]
8,"blow me, criticism IS constructive. \n\nBlow me Geoff.\n\nI came here to find out about monkeywr...","[toxic, obscene, insult]","[toxic, insult, obscene]"
9,"On account of the project deciding to ignore history, I hereby quit. Good bye. Oh, and Tim Vicke...","[toxic, obscene, insult]",[toxic]


# Case Study: Conference Title Classification

In [0]:
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Public/data/title_conference.csv
  
dbutils.fs.cp("file:/databricks/driver/title_conference.csv", "dbfs:/")

Out[152]: True

In [0]:
import pandas as pd
df = pd.read_csv('title_conference.csv')
df

Out[153]:

,Title,Conference
0,Innovation in Database Management: Computer Science vs. Engineering.,VLDB
1,High performance prime field multiplication for GPU.,ISCAS
2,enchanted scissors: a scissor interface for support in cutting and interactive fabrication.,SIGGRAPH
3,Detection of channel degradation attack by Intermediary Node in Linear Networks.,INFOCOM
4,Pinning a Complex Network through the Betweenness Centrality Strategy.,ISCAS
...,...,...
2502,A new QR-decomposition based recursive frequency estimator for multiple sinusoids in impulsive n...,ISCAS
2503,CNN Implementation of Spin Filters for Electronic Speckle Pattern Interferometry Applications.,ISCAS
2504,FaceKit: A Database Interface Design Toolkit.,VLDB
2505,On the trade-off between the number of scrolls and the operating frequency of the chaotic attrac...,ISCAS


In [0]:
df.Conference.value_counts()

Out[154]: ISCAS 864
INFOCOM 515
VLDB 423
WWW 379
SIGGRAPH 326
Name: Conference, dtype: int64

In [0]:
trainDataset = spark.read \
      .option("header", True) \
      .csv("/title_conference.csv")

(trainingData, testData) = trainDataset.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(trainingData.count()))
print("Test Dataset Count: " + str(testData.count()))


Training Dataset Count: 2030
Test Dataset Count: 477

In [0]:
document = DocumentAssembler()\
      .setInputCol("Title")\
      .setOutputCol("document")
    
# we can also use sentece detector here if we want to train on and get predictions for each sentence

use = UniversalSentenceEncoder.pretrained()\
      .setInputCols(["document"])\
      .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
      .setInputCols(["sentence_embeddings"])\
      .setOutputCol("class")\
      .setLabelColumn("Conference")\
      .setMaxEpochs(20)\
      .setEnableOutputLogs(True)

use_clf_pipeline = Pipeline(
    stages = [
        document,
        use,
        classsifierdl
    ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[ | ][OK!]

In [0]:
use_pipelineModel = use_clf_pipeline.fit(trainingData)

# 20 epochs takes around 25 seconds !

In [0]:
use_pipelineModel.stages

Out[158]: [DocumentAssembler_44c8c9038d33,
 UNIVERSAL_SENTENCE_ENCODER_4de71669b7ec,
 ClassifierDLModel_1218e0918f9c]

In [0]:
use_pipelineModel.stages[-1].write().overwrite().save('dbfs:/databricks/driver/models/use_clf')
use_clf_loaded = ClassifierDLModel.load('dbfs:/databricks/driver/models/use_clf')

use_clf_pipeline_pred = Pipeline(
    stages=[document, 
            use_pipelineModel.stages[-2],
            use_clf_loaded])

empty_data = spark.createDataFrame([[""]]).toDF("description")

result = use_clf_pipeline_pred.fit(empty_data).transform(testData)

In [0]:
result.select('Title','Conference',"class.result").show(10, truncate=80)


+--------------------------------------------------------------------------------+----------+----------+
 Title|Conference| result|
+--------------------------------------------------------------------------------+----------+----------+
 "From ""Think Like a Vertex"" to ""Think Like a Graph""."| VLDB|[SIGGRAPH]|
 1, 000 Tables Inside the From.| VLDB| [VLDB]|
 10Gbit/s 2mW inductorless transimpedance amplifier.| ISCAS| [ISCAS]|
3D reconstruction from planar points: A candidate method for authentication o...| ISCAS|[SIGGRAPH]|
 7-decades tunable translinear SiGe BiCMOS 3-phase sinusoidal oscillator.| ISCAS| [ISCAS]|
 A 0.18-&micro;m CMOS Squarer Circuit for a Non-Coherent UWB Receiver.| ISCAS| [ISCAS]|
 A 10-Bit 200-MS/s digitally-calibrated pipelined ADC using switching opamps.| ISCAS| [ISCAS]|
 A 10-bit 2GHz Current-Steering CMOS D/A Converter.| ISCAS| [ISCAS]|
 A 13-bit, low-power, compact ADC suitable for sensor applications.| ISCAS| [ISCAS]|
A 14-b 32MS/s Pipelined ADC with Novel Fast-convergence Comprehensive Backgro...| ISCAS| [ISCAS]|
+--------------------------------------------------------------------------------+----------+----------+
only showing top 10 rows

In [0]:
# We are going to use sklearn to evalute the results on test dataset
preds_df = result.select('Conference','Title',"class.result").toPandas()

# Let's explode the array and get the item(s) inside of result column out
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['Conference']))


precision recall f1-score support

 INFOCOM 0.65 0.71 0.68 90
 ISCAS 0.81 0.81 0.81 177
 SIGGRAPH 0.71 0.64 0.67 56
 VLDB 0.61 0.53 0.57 79
 WWW 0.61 0.67 0.64 75

 accuracy 0.70 477
 macro avg 0.68 0.67 0.67 477
weighted avg 0.70 0.70 0.70 477

In [0]:
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np

preds = pd.DataFrame(confusion_matrix(preds_df['result'], preds_df['Conference']), columns = np.unique(preds_df['Conference']), index =  np.unique(preds_df['Conference']))
preds

Out[162]:

,INFOCOM,ISCAS,SIGGRAPH,VLDB,WWW
INFOCOM,64,14,1,5,6
ISCAS,17,143,9,6,2
SIGGRAPH,1,9,36,4,6
VLDB,6,11,2,42,18
WWW,10,0,3,12,50


### Bert Sentence Embeddings

In [0]:
document = DocumentAssembler()\
              .setInputCol("Title")\
              .setOutputCol("document")
    
bert_sent = BertSentenceEmbeddings.pretrained('sent_small_bert_L8_512')\
              .setInputCols(["document"])\
              .setOutputCol("sentence_embeddings")

# the classes/labels/categories are in category column
classsifierdl = ClassifierDLApproach()\
              .setInputCols(["sentence_embeddings"])\
              .setOutputCol("class")\
              .setLabelColumn("Conference")\
              .setMaxEpochs(20)\
              .setLr(0.001)\
              .setBatchSize(8)\
              .setEnableOutputLogs(True)

bert_clf_pipeline = Pipeline(stages = [document,
                                      bert_sent,
                                      classsifierdl])

sent_small_bert_L8_512 download started this may take some time.
Approximate size to download 149.1 MB
[ | ][OK!]

In [0]:
classsifierdl.getMaxEpochs();

In [0]:
%%time
#20 epochs 30 secs !

bert_pipelineModel = bert_clf_pipeline.fit(trainingData)

CPU times: user 304 ms, sys: 98.7 ms, total: 403 ms
Wall time: 1min 12s

In [0]:
from sklearn.metrics import classification_report

preds = bert_pipelineModel.transform(testData)

preds_df = preds.select('Conference',"class.result").toPandas()

preds_df['result'] = preds_df['result'].apply(lambda x : x[0])

print (classification_report(preds_df['Conference'], preds_df['result']))

precision recall f1-score support

 INFOCOM 0.72 0.72 0.72 98
 ISCAS 0.85 0.88 0.86 177
 SIGGRAPH 0.69 0.67 0.68 51
 VLDB 0.59 0.68 0.63 69
 WWW 0.74 0.60 0.66 82

 accuracy 0.75 477
 macro avg 0.72 0.71 0.71 477
weighted avg 0.75 0.75 0.75 477

# Save model and Zip it for Modelshub Upload/Downloads

In [0]:
# Save a Spark NLP pipeline
bert_clf_pipeline.save('/databricks/driver/models/my_nlp_pipeline')

# cd into saved dir and zip
! cd ./my_nlp_pipeline ; zip -r my_nlp_pipeline.zip *

/bin/bash: line 0: cd: ./my_nlp_pipeline: No such file or directory
 adding: aclimdb_test.csv (deflated 60%)
 adding: aclimdb_test.csv.1 (deflated 60%)
 adding: aclimdb_train.csv (deflated 60%)
 adding: aclimdb_train.csv.1 (deflated 60%)
 adding: conf/ (stored 0%)
 adding: conf/spark-branch.conf (deflated 68%)
 adding: eventlogs/ (stored 0%)
 adding: eventlogs/1289542811931179064/ (stored 0%)
 adding: eventlogs/1289542811931179064/eventlog-2023-01-18--21-00.gz (deflated 5%)
 adding: eventlogs/1289542811931179064/eventlog-2023-01-18--22-00.gz (deflated 6%)
 adding: eventlogs/1289542811931179064/eventlog-2023-01-18--20-00.gz (deflated 0%)
 adding: eventlogs/1289542811931179064/eventlog-2023-01-18--23-00.gz (deflated 3%)
 adding: eventlogs/1289542811931179064/eventlog (deflated 95%)
 adding: ganglia/ (stored 0%)
 adding: ganglia/snapshot-2023-01-18T22:30:02.003Z.png (deflated 11%)
 adding: ganglia/snapshot-2023-01-18T22:15:01.755Z.png (deflated 11%)
 adding: ganglia/snapshot-2023-01-18T23:00:01.939Z.png (deflated 11%)
 adding: ganglia/snapshot-2023-01-18T22:45:01.891Z.png (deflated 12%)
 adding: logs/ (stored 0%)
 adding: logs/2023-01-18-21.log.gz (deflated 17%)
 adding: logs/2023-01-18-21.usage.json.gz (deflated 13%)
 adding: logs/stdout (deflated 72%)
 adding: logs/usage.json (deflated 97%)
 adding: logs/ttyd_logs (deflated 53%)
 adding: logs/stderr (deflated 73%)
 adding: logs/stdout--2023-01-18--22-00 (deflated 74%)
 adding: logs/stderr--2023-01-18--21-00 (deflated 87%)
 adding: logs/2023-01-18-22.log.gz (deflated 23%)
 adding: logs/2023-01-18-22.usage.json.gz (deflated 19%)
 adding: logs/2023-01-18-19.log.gz (deflated 9%)
 adding: logs/log4j-2023-01-18-22.log.gz (deflated 4%)
 adding: logs/log4j-2023-01-18-20.log.gz (deflated 7%)
 adding: logs/stderr--2023-01-18--23-00 (deflated 73%)
 adding: logs/stderr--2023-01-18--20-00 (deflated 77%)
 adding: logs/active.log (deflated 92%)
 adding: logs/2023-01-18-20.usage.json.gz (deflated 7%)
 adding: logs/log4j-active.log (deflated 93%)
 adding: logs/metrics.json (deflated 24%)
 adding: logs/stdout--2023-01-18--23-00 (deflated 70%)
 adding: logs/stderr--2023-01-18--22-00 (deflated 71%)
 adding: logs/2023-01-18-19.usage.json.gz (deflated 10%)
 adding: logs/lineage.json (stored 0%)
 adding: logs/2023-01-18-22.product.json.gz (deflated 14%)
 adding: logs/2023-01-18-21.product.json.gz (deflated 16%)
 adding: logs/stdout--2023-01-18--20-00 (deflated 88%)
 adding: logs/2023-01-18-22.metrics.json.gz (deflated 0%)
 adding: logs/2023-01-18-19.product.json.gz (deflated 14%)
 adding: logs/2023-01-18-20.metrics.json.gz (deflated 0%)
 adding: logs/2023-01-18-20.product.json.gz (deflated 13%)
 adding: logs/log4j-2023-01-18-19.log.gz (deflated 5%)
 adding: logs/2023-01-18-20.log.gz (deflated 12%)
 adding: logs/2023-01-18-19.metrics.json.gz (deflated 0%)
 adding: logs/2023-01-18-21.metrics.json.gz (deflated 0%)
 adding: logs/product.json (deflated 96%)
 adding: logs/log4j-2023-01-18-21.log.gz (deflated 6%)
 adding: logs/stdout--2023-01-18--21-00 (deflated 74%)
 adding: news_category_test.csv (deflated 61%)
 adding: news_category_test.csv.1 (deflated 61%)
 adding: news_category_test.csv.2 (deflated 61%)
 adding: news_category_train.csv (deflated 64%)
 adding: news_category_train.csv.1 (deflated 64%)
 adding: news_category_train.csv.2 (deflated 64%)
 adding: preload_class.lst (deflated 87%)
 adding: spam_ham_dataset.csv (deflated 65%)
 adding: spam_ham_dataset.csv.1 (deflated 65%)
 adding: spam_ham_dataset.csv.2 (deflated 65%)
 adding: title_conference.csv (deflated 64%)
 adding: title_conference.csv.1 (deflated 64%)
 adding: toxic_test.snappy.parquet (deflated 16%)
 adding: toxic_train.snappy.parquet (deflated 15%)

End of Notebook # 4